In [3]:
import numpy as np

In [112]:
def center(X, valid_frame_num):
    '''
    origin is moved to torso coordinates for each frame.
    '''
    # c - xyz, t - frames, v - join
    N, C, _, V = X.shape
    for i in range(N):
        for t in range(valid_frame_num[i]):
            # torso is the 3rd joint
            torso_coord = X[i, :, t, 2]
            for v in range(V):
                X[i, :, t, v] -= torso_coord
    return X

In [84]:
def dist_to_torso(joint_data, valid_frame_num):
    '''
    torso coords are always zero since we center relative to torso.
    in_shape = (3, n_frames)
    out_shape = (n_frames)
    '''
#     C, T = joint_data.shape
    dist_data = np.zeros(valid_frame_num)
    for t in range(valid_frame_num):
        dist_data[t] = np.linalg.norm(joint_data[:, t])   
    return dist_data

In [164]:
def dist_to_torso_allsamples(joint_data, valid_frame_num):
    '''
    torso coords are always zero since we center relative to torso.
    in_shape = (N, 3, n_frames)
    out_shape = (N, n_frames)
    '''
#     C, T = joint_data.shape
    N, _, T = joint_data.shape
    dist_data = np.zeros((N, T))
    for i in range(N):
        for t in range(valid_frame_num[i]):
            dist_data[i, t] = np.linalg.norm(joint_data[i, :, t])   
    return dist_data

In [166]:
def dist_to_torso_singleframe(joint_data):
    '''
    dist_to_torso for single frame
    '''
    return np.linalg.norm(joint_data)   

In [85]:
def dist_to_joint(joint1, joint2, valid_frame_num1, valid_frame_num2):
    '''
    in_shape = (3, n_frames) for both
    out_shape = (n_frames)
    '''
    assert(valid_frame_num1 == valid_frame_num2)
    dist_data = np.zeros(valid_frame_num1)
    for t in range(valid_frame_num1):
        dist_data[t] = np.linalg.norm(joint1[:, t] - joint2[:, t])
    return dist_data

In [168]:
def dist_to_joint_allsamples(joint1, joint2, valid_frame_num):
    '''
    in_shape = (N, 3, n_frames) for both
    out_shape = (N, n_frames)
    '''
    N, _, T = joint1.shape
    dist_data = np.zeros((N, T))
    for i in range(N):
        for t in range(valid_frame_num[i]):
            dist_data[i, t] = np.linalg.norm(joint1[i, :, t] - joint2[i, :, t])
    return dist_data

In [15]:
def dist_to_joint_single(joint1, joint2):
    '''
    dist_to_torso for single frame
    '''
    assert(len(joint1) == len(joint2))
    return np.linalg.norm(joint1 - joint2)

In [92]:
def flatten(X, valid_frame_num):
    '''
    turn two-dim data into one-dim data by stacking
    in_shape = (num_frames, features)
    out_shape = (valid_frames*features)
    '''
    _, V = X.shape
    data = np.zeros(valid_frame_num * V)
    for i in range(valid_frame_num):
        data[i*valid_frame_num : i*valid_frame_num + valid_frame_num] = X[i]
    return data

In [303]:
def cos_dist(joint1, joint2, valid_frame_num1, valid_frame_num2):
    '''
    cosine distance of a joint relative to another joint
    in_shape = (3, n_frames)
    out_shape = (3, n_frames)
    '''
    assert(valid_frame_num1 == valid_frame_num2)
    
    out = np.zeros(valid_frame_num1)
    for t in range(valid_frame_num1):
        dot_prod = np.dot(joint1[:, t], joint2[:, t])
        a = np.linalg.norm(joint1[:,t])
        b = np.linalg.norm(joint2[:, t])
        out[t] = dot_prod/(a * b)   
    return out

In [315]:
def cos_dist_allsamples(joint1, joint2, valid_frame_num):
    '''
    cosine distance of a joint relative to another joint, For Torso requires non-centered data for both joints
    in_shape = (N, 3, n_frames)
    out_shape = (N, 3, n_frames)
    '''
    
    N, _, T = joint1.shape
    out = np.zeros((N, T))
    for i in range(N):
        for t in range(valid_frame_num[i]):
            dot_prod = np.dot(joint1[i, :, t], joint2[i, :, t])
            a = np.linalg.norm(joint1[i, :, t])
            b = np.linalg.norm(joint2[i, :, t])
            out[i, t] = dot_prod/(a * b)   
            print(out[i, t])
    return out

In [89]:
def horizontal_flip(X, valid_frame_num):
    '''
    flip horizontally all data relative to torso, assumes that data is centered at torso
    in_shape = out_shape
    '''
    for t in range(valid_frame_num):
        X[0, t, :] *= -1

In [104]:
def clip_samples(X, valid_frames_num):
    '''
    make sequnces the same length equal to the shorted sequence
    '''
    min_seq = min(valid_frames_num)
    return X[:, :, :min_seq, :], min_seq

In [142]:
def cut_sample(X, valid_frame_num, length):
    '''
    takes in data and returns valid_frame_num/length samples
    in_shape = (xyz, n_frames, joints)
    '''
    times = int(valid_frame_num/length)
    samples = []
    for i in range(times):
        idx_s = i * length
        idx_e = i * length + length
        s = X[:, idx_s:idx_e, :]
        samples.append(s)
    return samples, times

In [91]:
def sample_frames_from_sequnce(X):
    '''
    assumes sample is cut already
    in_shape = (num_frames, features)
    takes two-dim data and reduced the frames
    out_shape = (9, features)
    '''
    T, V = X.shape
    # for now implemented for 45
    assert(T == 45)
    idx = [0, 4, 9, 14, 19, 24, 29, 34, 39, 44]
    data = np.zeroes((9, V))
    for i in range(9):
        data[i] = X[idx[i], :]

In [72]:
from feeder import feeder

In [251]:
data_path = "data0/CAD-60/train_data.npy"
label_path = "data0/CAD-60/train_label.pkl"
num_frame_path = "data0/CAD-60/train_num_frame.npy"
dataset = feeder.Feeder(data_path, label_path, num_frame_path)

In [254]:
X = dataset.data
Y = dataset.label
valid_frame_num = dataset.valid_frame_num

In [255]:
# clip sequences
X, new_len = clip_samples(X, valid_frame_num)
valid_frame_num = np.full(len(valid_frame_num), new_len)

In [256]:
N, C, T, V = X.shape
print(N)
print(C)
print(T)
print(V)

60
3
147
15


In [257]:
new_data = np.zeros((N*3, C, length, V))
new_label = np.zeros(N*3)
length = 45
for i in range(N):
    sample = X[i, :, :, :]
    label = Y[i]
    samples, times = cut_sample(sample, valid_frame_num[i], length)
    for j in range(times):
        new_data[i*times + j, :, :, :] = samples[j]
        new_label[i*times + j] = label
valid_frame_num = np.full(len(valid_frame_num)*3, length)

In [259]:
X_uncentered = np.copy(new_data)

In [261]:
X_uncentered[0, :, :, 2]

array([[ 218.30000305,  216.30000305,  215.5       ,  214.5       ,
         212.8999939 ,  214.1000061 ,  213.3999939 ,  213.69999695,
         216.3999939 ,  215.30000305,  214.1000061 ,  214.        ,
         215.19999695,  215.3999939 ,  215.69999695,  214.        ,
         212.30000305,  211.6000061 ,  210.        ,  212.3999939 ,
         213.30000305,  210.5       ,  210.30000305,  209.6000061 ,
         210.69999695,  212.30000305,  214.5       ,  215.3999939 ,
         216.69999695,  217.80000305,  216.80000305,  216.6000061 ,
         218.30000305,  220.8999939 ,  223.6000061 ,  224.19999695,
         224.30000305,  224.8999939 ,  224.1000061 ,  224.8999939 ,
         225.19999695,  225.30000305,  223.        ,  221.69999695,
         220.80000305],
       [ 124.19999695,  125.09999847,  125.40000153,  124.30000305,
         122.09999847,  121.90000153,  121.        ,  121.09999847,
         118.19999695,  120.09999847,  120.        ,  120.40000153,
         119.30000305,  

In [262]:
X = new_data
Y = new_label
N, C, T, V = X.shape
print(X.shape)
print(Y.shape)

(180, 3, 45, 15)
(180,)


In [263]:
# center at torso
X = center(X, valid_frame_num)

In [264]:
X[0, :, :, 2]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [265]:
X_uncentered[0, :, :, 2]

array([[ 218.30000305,  216.30000305,  215.5       ,  214.5       ,
         212.8999939 ,  214.1000061 ,  213.3999939 ,  213.69999695,
         216.3999939 ,  215.30000305,  214.1000061 ,  214.        ,
         215.19999695,  215.3999939 ,  215.69999695,  214.        ,
         212.30000305,  211.6000061 ,  210.        ,  212.3999939 ,
         213.30000305,  210.5       ,  210.30000305,  209.6000061 ,
         210.69999695,  212.30000305,  214.5       ,  215.3999939 ,
         216.69999695,  217.80000305,  216.80000305,  216.6000061 ,
         218.30000305,  220.8999939 ,  223.6000061 ,  224.19999695,
         224.30000305,  224.8999939 ,  224.1000061 ,  224.8999939 ,
         225.19999695,  225.30000305,  223.        ,  221.69999695,
         220.80000305],
       [ 124.19999695,  125.09999847,  125.40000153,  124.30000305,
         122.09999847,  121.90000153,  121.        ,  121.09999847,
         118.19999695,  120.09999847,  120.        ,  120.40000153,
         119.30000305,  

In [266]:
# new array for features
features = np.zeros((N, T, 11))

In [316]:
# left elbow, right elbow, left foot, right foot)
joints_to_torso = [4, 6, 13, 14]
stop = 0
for i in range(len(joints_to_torso)):
    features[:, :, i] = dist_to_torso_allsamples(X[:, :, :, joints_to_torso[i]], valid_frame_num)
    stop += 1
# left hand, right hand
joints_to_face = [11, 12]
for i in range(len(joints_to_face)):
    features[:, :, stop + i] = dist_to_joint_allsamples(X[:, :, :, joints_to_face[i]], X[:, :, :, 0], valid_frame_num)
    stop += 1
# left hand, right hand, head, left knee, right knee
angle_totorso = [11, 12, 0, 8, 9]
for i in range(len(angle_totorso)):
    # need uncentered data here
    features[:, :, stop + i] = cos_dist_allsamples(X_uncentered[:, :1, :, angle_totorso[i]], X_uncentered[:, :1, :, 2], valid_frame_num)

-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0


1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-1.0
-1.0
-1.0
-1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


...

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-1.0
-1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0


1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1

In [278]:
a = X_uncentered[3, :, :, 11]
b = X_uncentered[3, :, :, 2]

In [279]:
a.shape

(3, 45)

In [280]:
b.shape

(3, 45)

In [281]:
a

array([[  48.70000076,   47.59999847,   47.        ,   44.20000076,
          43.09999847,   38.5       ,   34.5       ,   34.40000153,
          33.40000153,   32.59999847,   31.89999962,   30.70000076,
          30.20000076,   30.        ,   30.20000076,   33.        ,
          40.        ,   45.90000153,   49.90000153,   51.5       ,
          53.        ,   60.79999924,   66.59999847,   69.30000305,
          72.19999695,   74.        ,   77.40000153,   79.09999847,
          80.        ,   80.80000305,   83.19999695,   84.5       ,
          86.40000153,   88.19999695,   89.        ,   87.40000153,
          87.19999695,   86.19999695,   89.09999847,   89.30000305,
          90.59999847,   94.5       ,   96.5       ,   96.09999847,
          94.90000153],
       [ 315.20001221,  310.79998779,  300.5       ,  283.20001221,
         275.8999939 ,  260.29998779,  238.6000061 ,  220.        ,
         202.6000061 ,  180.69999695,  167.69999695,  151.3999939 ,
         139.80000305,  

In [282]:
b

array([[ -98.19999695,  -98.19999695,  -97.19999695,  -94.19999695,
         -93.19999695,  -95.69999695,  -93.09999847,  -91.30000305,
         -87.09999847,  -89.90000153,  -92.59999847,  -91.09999847,
         -89.59999847,  -92.19999695,  -93.40000153,  -94.19999695,
         -94.        ,  -93.59999847,  -92.09999847,  -92.40000153,
         -89.90000153,  -88.69999695,  -87.69999695,  -82.        ,
         -78.19999695,  -75.40000153,  -72.5       ,  -70.09999847,
         -63.5       ,  -59.70000076,  -56.90000153,  -54.5       ,
         -50.09999847,  -52.70000076,  -52.        ,  -47.40000153,
         -44.        ,  -42.5       ,  -38.70000076,  -36.40000153,
         -33.79999924,  -36.        ,  -35.29999924,  -35.79999924,
         -36.20000076],
       [  78.09999847,   75.90000153,   76.69999695,   74.19999695,
          72.59999847,   74.5       ,   73.        ,   75.80000305,
          77.69999695,   78.30000305,   77.09999847,   75.59999847,
          72.59999847,  

In [286]:
cos_dist(a, b, 45, 45)

array([0.98799915, 0.98821693, 0.98905638, 0.99025648, 0.99069812,
       0.99182594, 0.99318521, 0.99430627, 0.99531964, 0.99609444,
       0.99641448, 0.99692309, 0.99720524, 0.99724338, 0.99728335,
       0.99695073, 0.9964977 , 0.99609199, 0.99565002, 0.99522766,
       0.99455922, 0.99340798, 0.99262606, 0.99207638, 0.99143565,
       0.99077941, 0.99005532, 0.98938503, 0.98868313, 0.98857793,
       0.98845204, 0.98827148, 0.98779589, 0.98719851, 0.98676733,
       0.98763815, 0.98811182, 0.9883999 , 0.98804073, 0.98790473,
       0.9879591 , 0.98701508, 0.9871331 , 0.98693124, 0.98656125])

In [289]:
c = X_uncentered[3, :, :, 0]
c

array([[-3.79000015e+01, -3.69000015e+01, -3.57000008e+01,
        -3.42000008e+01, -3.25000000e+01, -2.96000004e+01,
        -2.81000004e+01, -2.58999996e+01, -2.32000008e+01,
        -2.07999992e+01, -1.93999996e+01, -1.70000000e+01,
        -1.82000008e+01, -1.77000008e+01, -1.58999996e+01,
        -1.75000000e+01, -1.93999996e+01, -1.96000004e+01,
        -1.92999992e+01, -1.81000004e+01, -1.75000000e+01,
        -1.48000002e+01, -1.35000000e+01, -9.69999981e+00,
        -8.30000019e+00, -6.90000010e+00, -5.40000010e+00,
        -1.39999998e+00,  1.89999998e+00,  7.30000019e+00,
         1.06999998e+01,  1.46000004e+01,  1.62999992e+01,
         1.78999996e+01,  2.06000004e+01,  2.33999996e+01,
         2.58999996e+01,  2.68999996e+01,  2.55000000e+01,
         2.47999992e+01,  2.63999996e+01,  2.71000004e+01,
         2.77000008e+01,  2.57999992e+01,  2.81000004e+01],
       [ 4.72100006e+02,  4.69899994e+02,  4.70799988e+02,
         4.68399994e+02,  4.66200012e+02,  4.66700012e+

In [291]:
cos_dist(b, c, 45, 45)

array([0.98014658, 0.98019336, 0.98021117, 0.98029974, 0.98042241,
       0.98049819, 0.9804919 , 0.98049341, 0.9805137 , 0.98049912,
       0.98054041, 0.9805078 , 0.98050268, 0.98050232, 0.98033132,
       0.980316  , 0.98033484, 0.98030926, 0.98016614, 0.98005511,
       0.98005703, 0.98008168, 0.98007651, 0.98021078, 0.98007421,
       0.98011294, 0.98015243, 0.98012275, 0.98017409, 0.98029564,
       0.98036234, 0.98038761, 0.98063503, 0.98069779, 0.98083607,
       0.98071392, 0.98046693, 0.98033995, 0.98035935, 0.98031291,
       0.98092504, 0.98055104, 0.98044453, 0.98131959, 0.98161502])

In [338]:
d = X_uncentered[3, :, :, 11]
d.shape

(3, 45)

In [293]:
cos_dist(d, b, 45, 45)

array([0.98217541, 0.98222058, 0.98186093, 0.98247755, 0.98302451,
       0.98351765, 0.98385788, 0.98324929, 0.98292932, 0.98268402,
       0.98268144, 0.98253101, 0.98250133, 0.98253117, 0.98273635,
       0.98271908, 0.98243621, 0.98276675, 0.98297364, 0.98320319,
       0.98313358, 0.98343936, 0.98368218, 0.98363069, 0.98382535,
       0.98403672, 0.98459393, 0.98564699, 0.9854569 , 0.98516328,
       0.985273  , 0.9852252 , 0.98774829, 0.99045001, 0.99169851,
       0.99031034, 0.98865737, 0.98741299, 0.98643169, 0.98638629,
       0.98858926, 0.99147811, 0.98940485, 0.99046771, 0.99193334])

In [298]:
d[:, 0]

array([  89.59999847, -272.1000061 , 2148.80004883])

In [339]:
out =[]
for i in range(5):
    print(d[:, i])
    print('-----')
    print(b[:, i])
    print('****')
    print(np.dot(d[:1, i], b[:1, i]))
    print('.....')
    print(np.linalg.norm(d[:1, i]), np.linalg.norm(b[:1, i]))
    print('~~~~~~')
    print(np.dot(d[:1, i], b[:1, i])/(np.linalg.norm(d[:1, i]) * np.linalg.norm(b[:1, i])))

[  48.70000076  315.20001221 1787.5       ]
-----
[ -98.19999695   78.09999847 1965.        ]
****
-4782.3399263000465
.....
48.70000076293945 98.19999694824219
~~~~~~
-1.0
[  47.59999847  310.79998779 1787.69995117]
-----
[ -98.19999695   75.90000153 1964.59997559]
****
-4674.319704895024
.....
47.599998474121094 98.19999694824219
~~~~~~
-1.0
[  47.   300.5 1787.5]
-----
[ -97.19999695   76.69999695 1963.90002441]
****
-4568.399856567383
.....
47.0 97.19999694824219
~~~~~~
-1.0
[  44.20000076  283.20001221 1785.30004883]
-----
[ -94.19999695   74.19999695 1964.30004883]
****
-4163.639936981199
.....
44.20000076293945 94.19999694824219
~~~~~~
-1.0
[  43.09999847  275.8999939  1784.40002441]
-----
[ -93.19999695   72.59999847 1965.90002441]
****
-4016.919726257329
.....
43.099998474121094 93.19999694824219
~~~~~~
-1.0


In [341]:
body = X_uncentered[34, :, :, 2]
left_knee = X_uncentered[34, :, :, 8]
right_knee = X_uncentered[34, :, :, 9]

In [336]:
name = dataset.sample_name
name[34]

'/data3/0511130523.txt'

In [346]:
for i in range(5):
    print(body[:, i])
    print('-----')
    print(right_knee[:, i])
    print('****')
    print(np.dot(body[:1, i], right_knee[:1, i]))
    print('.....')
    print(np.linalg.norm(body[:1, i]), np.linalg.norm(right_knee[:1, i]))
    print('~~~~~~')
    print(np.dot(body[:1, i], right_knee[:1, i])/(np.linalg.norm(body[:1, i]) * np.linalg.norm(right_knee[:1, i])))

[ 155.8999939 -302.1000061 2482.       ]
-----
[ 218.3999939  -420.70001221 2308.89990234]
****
34048.55771545414
.....
155.89999389648438 218.39999389648438
~~~~~~
1.0
[ 156.1000061  -302.20001221 2483.5       ]
-----
[ 218.6000061  -421.20001221 2310.60009766]
****
34123.46228698734
.....
156.10000610351562 218.60000610351562
~~~~~~
1.0
[ 151.30000305 -309.20001221 2474.19995117]
-----
[ 212.5 -427.  2300. ]
****
32151.250648498535
.....
151.3000030517578 212.5
~~~~~~
1.0
[ 151.         -305.6000061  2480.10009766]
-----
[ 211.1000061  -424.70001221 2306.5       ]
****
31876.10092163086
.....
151.0 211.10000610351562
~~~~~~
1.0
[ 151.19999695 -303.29998779 2484.39990234]
-----
[ 210.5        -419.79998779 2308.80004883]
****
31827.59935760498
.....
151.1999969482422 210.5
~~~~~~
1.0
